In [23]:
def parse_params_entry(row, columns):

    """
    zmienna columns dostaje kolejne nazwy do bardziej "globalnej" zmiennej
    a zwracana jest lista z wartościami kolejnej obserwacji
    """

    entry = []
    unordered = {} # wartości początkowo nie są posortowane bo stopniowo są dodawane nazwami a nie kolejnością
    for param in row.split("<br>"):
        
        key_value = param.split("<=>")

        # pomija parametry bez wartości
        no_value = len(key_value) == 1
        if (no_value):
            continue

        key = key_value[0]

        # niektóre zmienne kończą się na _types i mają wtedy wiele możliwości które się dodatkowo łączą
        # tutaj są zamieniane na zmienne binarne
        if (key.endswith("types")):
            
            if key_value[1].strip() == "" or key_value[1].strip() == "0":
                continue

            for value in key_value[1].split("<->"):

                real_key = key + "_" + value
                if real_key not in columns:
                    columns.append(real_key)
                unordered[real_key] = True
            continue
        
        # dodaje nazwę parametru do nagłówka
        value = key_value[1]
        if key not in columns:
            columns.append(key)

        unordered[key] = value

    for key in columns:
        entry.append(unordered[key] if key in unordered else None)    

    return entry

In [24]:
def read_data(path, param_name = "params", nchunks = None, drop = [], chunksize = 10000):

    """
    zbiera parametry w formacie <key><=>value<->value<->value<br>key><=>value<->... z DataFrame,
    >>> parse_params(df)
    id    price[currency]       m rooms_num     market  ... fence_types heating_types access_types vicinity_types is_bungalow
    325017             PLN   72.14         4  secondary  ...        None          None         None           None        None
    drop określa które kolumny wykluczyć ze wczytywania
    """

    import pandas

    param_cols = []
    csv_entries = []

    print('Ładowanie wierszy:')
    empty = pandas.read_csv(path, nrows=0).columns.drop(drop)
                                                 # drop wybrane kolumny
    
    # iterator sprawia że w trakcie ładowania pliku przeprowadzane są od razu działania
    iterator = pandas.read_csv(path, chunksize=chunksize, usecols=empty)
                                                        # usecols sprawia że tylko wybrane kolumny są wczytane
    for (k, chunk) in enumerate(iterator):

        if (nchunks is not None and k >= nchunks):
            break      
        for (i, row) in chunk.iterrows():
            
            entry = parse_params_entry(row[param_name], param_cols)
            csv_entries.append([*row.drop(param_name), *entry])

        print(f'{(k+1)*10}k', end='\n' if ((k+1)%10 == 0) else ' ')
    
    from pandas import DataFrame

    return DataFrame(csv_entries, columns=[*empty.drop(param_name), *param_cols])

In [25]:
df = read_data('train.csv', drop = ['description', 'title'])
                          # drop - pomijamy kolumny tekstowe
df.set_index('id', inplace=True)

Ładowanie wierszy:
10k 20k 30k 40k 50k 60k 70k 80k 90k 100k
110k 120k 130k 140k 150k 160k 170k 180k 190k 200k
210k 220k 230k 240k 250k 260k 270k 280k 290k 300k
310k 320k 330k 340k 350k 360k 370k 380k 390k 400k
410k 420k 430k 440k 450k 460k 470k 480k 490k 500k
510k 520k 530k 540k 550k 560k 570k 580k 590k 600k
610k 620k 630k 640k 650k 660k 670k 680k 690k 700k
710k 720k 730k 740k 750k 760k 770k 780k 790k 800k
810k 820k 830k 840k 

In [26]:
def get_lon_lat_inplace(df, type = 'city', path = 'cities.csv'):
    
    import pandas
    
    cities = pandas.read_csv(path)[['id','lon','lat']]
    cities.rename(columns= {"id" : f'{type}_id', 'lon': f'{type}_lon', 'lat': f'{type}_lat'}, inplace=True)
    
    df = pandas.merge(df, cities, on = f'{type}_id', how='left')

get_lon_lat_inplace(df, 'city', 'cities.csv')
get_lon_lat_inplace(df, 'district', 'districts.csv')

In [27]:
df.drop(columns=['city_id', 'region_id', 'district_id'], inplace=True)

In [28]:
df.to_csv('dataset.csv')